In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [8]:
# import sys
# !{sys.executable} -m pip install tensorflow-gpu 

In [9]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [11]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from keras.wrappers.scikit_learn import KerasRegressor
from keras import Sequential
from keras.layers import Dense

# from tsfresh.examples import load_robot_execution_failures
# from tsfresh import extract_features, select_features
import optuna

# from common import EP
# from models import *
# from dfdb import DFDB

import types
import os
import copy

import seaborn as sns
import matplotlib.pyplot as plt

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="3";  

In [14]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if not f.startswith('.')]

['struct_eigen.pkl',
 'pytorch_geometric_test.pkl',
 'keras-neural-net-for-champs_train_charge.pkl',
 'keras-neural-net-for-champs_testelectronegativity.pkl',
 'distance-is-all-you-need_test.gzde',
 'laplaction_matrix_test.pkl',
 'tmp_df_train',
 'test_cis_trans.csv',
 'df_train2.gzde',
 'pytorch_geometric_train.pkl',
 'laplaction_matrix_train.pkl',
 'knn_.pkl',
 'df_train.gzde',
 'df_test2.gzde',
 'distance-is-all-you-need_train.gzde',
 'keras-neural-net-for-champs_trainelectronegativity.pkl',
 'train_cis_trans.csv',
 'df_test.gzde',
 'keras-neural-net-for-champs_test_charge.pkl',
 'tmp_df_test']

In [15]:
df_train=pd.read_pickle(f'{file_folder}/tmp_df_train', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/tmp_df_test', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [23]:
def create_path(base_dir, param):
    if base_dir == None:
        return None
    fold_path = base_dir + '/' + ','.join("{!s}={!r}".format(key,val) for (key,val) in param.items())
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
    return fold_path

path_param={
    'hidden_layer_sizes':[512, 128, 16],
    'activation':'relu',
    'l1l2regularizer':None,
    'dropout':.3,
}
base_save_dir = create_path('KerasMLPRegressor', path_param)
param={
    'columns': [],
 'cv': {'cls': 'KFold',
  'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}},
 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
 'model': {'cls': 'KerasMLPRegressor',
  'init': {'input_dim':0,
            'batch':128, 
            'solver':'adam', 
            'metric':'mean_absolute_error', 
            'lr':.0001, 
            'sgd_momentum':.9, 
            'sgd_decay':0.0001,
            'base_save_dir':base_save_dir, 
            'alias':'kerasmlp',
            **path_param},
  'fit':  {'versbose':0, 
            'epochs':50, 
            'eval_set':{}},},
 'metric': 'mean_absolute_error'
}
    

In [24]:
columns_list = [
{'columns': ['tertiary_distance_2', 'dist_H_2_y', 'dist_H_0_y', 'distC0', 'inv_dist1R', 'dist_C_2_x', 'dist_to_type_std', 'dist_C_2_y', 'dist_C_0_x', 'atom_1_n_bonds', 'd_5_1', 'yukawa_H.y', 'd_4_1', 'd_4_2', 'd_4_3', 'd_3_0', 'dist_H_2_x', 'd_4_0', 'tertiary_distance_4', 'tertiary_angle_1', 'd_3_1', 'dist_H_1_y', 'coulomb_H.y', 'dist_C_4_x', 'atom_index_farthest_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_c1', 'atom_1_bond_lengths_std', 'inv_dist0R', 'eem_0', 'inv_distPR', 'dist_N_0_y', 'dist_H_3_x', 'distC1', 'tertiary_atom_0', 'dist_H_1_x', 'dist_C_0_y', 'adC1', 'max_distance_y', 'tertiary_distance_5', 'dist_to_type_0_mean', 'atom_1_bond_lengths_mean', 'd_1_0', 'dist_O_1_y', 'tertiary_angle_0', 'yukawa_N.x', 'dist_C_4_y', 'atom_index_1_cycle_size_mean', 'tertiary_atom_1', 'inv_dist1', 'd_2_1', 'dist_C_1_y', 'adC2', 'inv_dist1E', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_1_mean', 'dist_to_type_mean', 'adC3', 'dist_O_0_y', 'eem_1', 'adN1', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'dist_O_1_x', 'dist_C_3_y', 'tertiary_angle_3', 'cos_f0', 'd_3_2', 'dist_O_0_x', 'dist_H_3_y', 'dist_C_3_x', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['yukawa_N.x', 'inv_distPR', 'molecule_atom_index_0_dist_mean_div', 'dist_C_0_y', 'dist_N_1_x', 'dist_C_1_x', 'd_4_1', 'molecule_atom_1_dist_min_diff', 'dist_N_0_y', 'cos_f1', 'dist_O_0_x', 'tertiary_distance_5', 'vander_N.x', 'dist_C_3_x', 'coulomb_H.x', 'dist_C_3_y', 'd_9_2', 'distance_center0', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'dist_to_type_0_mean', 'dist_N_0_x', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_std', 'd_3_1', 'dist_H_3_y', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'tertiary_distance_0', 'linkN', 'd_2_0', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'tertiary_angle_0', 'dist_N_1_y', 'adC3', 'inv_dist1R', 'dist_H_1_y', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'd_2_1', 'dist_H_2_y', 'tertiary_distance_4', 'link0', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_0', 'yukawa_H.y', 'molecule_name.1', 'distance_c1', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'cos_center0', 'cos_center0_center1', 'inv_dist1', 'eem_0', 'vander_H.x', 'tertiary_distance_2', 'dist_C_4_x', 'distN0', 'dist_H_0_x', 'max_distance_y', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['molecule_atom_index_1_dist_min_diff', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'atom_8', 'd_4_1', 'd_9_1', 'dist_N_0_y', 'cos_c1', 'd_7_0', 'dist_O_0_x', 'tertiary_distance_5', 'dist_C_3_x', 'dist_O_1_x', 'dist_C_3_y', 'd_7_2', 'd_3_2', 'd_5_2', 'd_6_0', 'dist_C_2_x', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_max_diff', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'd_3_1', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'tertiary_angle_3', 'atom_4', 'tertiary_distance_0', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'bond_atom', 'd_8_2', 'tertiary_angle_0', 'distC0', 'adC3', 'inv_dist1R', 'd_8_1', 'd_7_1', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'atom_index_1_cycle_size_mean', 'inv_dist0', 'd_2_1', 'dist_H_2_y', 'atom_1_n_bonds', 'tertiary_angle_2', 'd_1_0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_2_y', 'tertiary_atom_1', 'vander_O.y', 'tertiary_atom_0', 'distC1', 'adC2', 'atom_7', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'dist_C_4_y', 'd_5_1', 'molecule_type_dist_max', 'd_6_1', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_5', 'dist_H_0_x', 'cos_c0', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['molecule_atom_index_0_dist_min_diff', 'inv_distPR', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'd_4_1', 'dist_N_0_y', 'cos_c1', 'dist_to_type_mean', 'yukawa_H.x', 'cos_f1', 'dist_O_0_x', 'dist_H_3_x', 'dist_C_3_x', 'dist_C_3_y', 'd_7_2', 'dist_O_1_y', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'inv_distPE', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_dist_min', 'dist_O_0_y', 'd_4_3', 'dist_to_type_1_mean', 'dist_H_1_x', 'd_3_1', 'dist_H_3_y', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'adC4', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'tertiary_angle_0', 'yukawa_C.x', 'mean_molecule_atom_0_dist_xyz', 'adC3', 'dist_H_1_y', 'dist_H_2_x', 'dist_C_1_y', 'tertiary_distance_3', 'coulomb_H.y', 'd_2_1', 'dist_H_2_y', 'vander_H.y', 'link0', 'tertiary_distance_4', 'vander_C.x', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_1', 'yukawa_H.y', 'dist_H_4_y', 'mean_molecule_atom_1_dist_xyz', 'adC2', 'd_5_1', 'molecule_type_dist_max', 'vander_N.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'vander_H.x', 'tertiary_distance_2', 'atom_5', 'cos_c0', 'vander_O.x', 'cos_f0_f1', 'dist_H_0_x', 'max_distance_y', 'dist_xyz', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_1_mean', 'cos_f0', 'dist_O_0_x', 'atom_1_bond_lengths_min', 'd_5_2', 'cos_c1', 'd_2_1', 'molecule_atom_index_0_dist_min_div', 'atom_index_1_cycle_size_mean', 'd_4_2', 'inv_distPR', 'dist_N_0_x', 'dist_O_0_y', 'adC1', 'adC2', 'tertiary_angle_2', 'dist_C_2_y', 'molecule_atom_index_0_dist_std_diff', 'dist_C_2_x', 'dist_C_3_y', 'd_4_3', 'eem_1', 'molecule_atom_index_1_dist_min_diff', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'inv_dist0R', 'd_4_0', 'd_4_1', 'mean_molecule_atom_0_dist_xyz', 'd_2_0', 'd_3_1', 'dist_C_1_x', 'dist_H_0_y', 'dist_C_0_y', 'd_5_3', 'tertiary_distance_3', 'd_3_2', 'dist_H_1_x', 'tertiary_distance_1', 'atom_1_n_bonds', 'd_8_3', 'cos_c0', 'tertiary_distance_2', 'tertiary_angle_0', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'yukawa_H.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
]

In [25]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    
    
    columns = columns_list[t]['columns']
    param['model']['init']['input_dim'] = len(columns)
    param['columns'] = columns
    
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), param, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=0)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)
    break

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
# df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [26]:
df_trial = pd.DataFrame(mytrial)
df_trial['trn_val_metric_diff_rate'] = df_trial['trn_val_metric_diff'] / df_trial['train_metric_mean']
df_trial['log_val_mae'] = df_trial['val_metric_mean'].apply(lambda x : np.log(x))
# print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message', 'log_val_mae']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,trn_val_metric_diff_rate,message,log_val_mae
0,2019-08-17 17:12:44.726904,modeling for 0,74,1.848263,1.859721,0.011458,0.006199,modeling for 0,0.620427


In [22]:
len(columns)

74